<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_13/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_LSTM_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Реализация LSTM
В этом уроке мы рассмотрим реализацию продвинутого RNN слоя -- LSTM. Как и раньше, создадим одну версию с помощью Keras и одну версию с помощью чистого TensorFlow (для наглядной демонстрации внутреннего устройства).

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [2]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.8.2


### Входной тензор
Подготовим входной тензор для тестов аналогично тому, как мы это делали в предыдущем уроке.

Размерности тензора `x`: (батч, длина цепочки, размер эмбеддинга)

In [12]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)

x = np.random.rand(BATCH_SIZE, SEQ_LEN, EMB_SIZE).astype(np.float32)
print(x.shape)

(2, 100, 16)


### Создание простого LSTM слоя с помощью Keras
Создание LSTM слоя на Keras очень похоже на создание SimpleRNN слоя

`H_SIZE` - количество элементов в выходном векторе `h`. Такое же количество элементов будет в вектор внутренней памяти `C`.

`return_sequences` - флаг, в зависимости от которого на выходе мы получаем цепочку векторов `h` (той же длины что и входная цепочка) или только последний вектор `h`

`recurrent_activation` - функция активации для некоторых внутренних полносвязных слоёв. В классической LSTM это sigmoid

In [13]:
H_SIZE = 32

lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, recurrent_activation='sigmoid')

Запускаем инференс (прямое распространение) для созданного слоя и проверяем размерность выходного тензора.

Она должна была получиться `(BATCH_SIZE, SEQ_LEN, H_SIZE)`

In [14]:
y = lstm(x)
print(y.shape)

(2, 100, 32)


### Реализация LSTM ячейки
Ранее мы обсуждали, что в рекуррентных сетях имеет смысл выделять понятие "ячейка". Это по сути слой, который работает с текущим элементом последовательности. А уже внешний рекуррентный слой использует ячейку для прохода по последовательности.

Для LSTM ячейки задача состоит в том, чтобы по входному вектору `x`, скрытому состоянию `h` и вектору памяти `c` предсказать новые `h` и `c`.

В основе LSTM ячейки лежат 4 линейных преобразования: 3 гейта (input, forget, output) и преобразование, ответственное за создание нового кандидата в ячейку памяти.

Каждое такое линейное преобразование задаётся двумя матрицами и одним вектором смещения (как и в SimpleRNN), поэтому для его описания нужно два полносвязных слоя, у одного из которых выключен биас.

Прямое распространение (в функции `call`) осуществляется согласно оригинальному алгоритму LSTM (из теоретической части).


In [15]:
class LSTMCell(tf.keras.Model):
    def __init__(self, h_size):
        super().__init__()
        self.h_size = h_size
        
        # input gate
        self.fcXI = tf.keras.layers.Dense(self.h_size)
        self.fcHI = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
        # forget gate
        self.fcXF = tf.keras.layers.Dense(self.h_size)
        self.fcHF = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
        # создание нового кандидата CH
        self.fcXC = tf.keras.layers.Dense(self.h_size)
        self.fcHC = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
        # output gate
        self.fcXO = tf.keras.layers.Dense(self.h_size)
        self.fcHO = tf.keras.layers.Dense(self.h_size, use_bias=False)
        
    def call(self, x, h, c):
        i = tf.nn.sigmoid(self.fcXI(x) + self.fcHI(h))
        f = tf.nn.sigmoid(self.fcXF(x) + self.fcHF(h))
        o = tf.nn.sigmoid(self.fcXO(x) + self.fcHO(h))
        ch = tf.nn.tanh(self.fcXC(x) + self.fcHC(h))
        c = f*c + i*ch # поэлементное произведение и сумма
        h = o*tf.nn.tanh(c) # поэлементное произведение
        return h, c

**[Задание 1]** Реализуйте LSTM слой (класс `LSTM`, наследованный от `tf.keras.Model`), эквивалентный описанной выше модели на Keras. Используйте для этого описанную выше ячейку `LSTMCell`. 

**[Задание 2]** Сравните результаты работы класса нового `LSTM` с оригинальной моделью на Keras так же, как мы это делали раньше (разница должна получиться равной нулю или очень маленькой). 

Вам также будет необходимо скопировать веса в новую модель. В модели Keras веса находятся вот в таких переменных:

`lstm.weights[0]` - матрицы, связанные с вектором x

`lstm.weights[1]` - матрицы, связанные с вектором h

`lstm.weights[2]` - биасы

В каждой такой матрице объединены все матрицы, соответствующие разным линейным преобразованиям.

Их индексы подматриц:

`I` -- 0:32

`F` -- 32:64

`C` -- 64:96

`O` -- 96:128

Например, матрица весов (`kernel`) для слоя `fcHC` будет находиться в подматрице `lstm.weights[1][:, 64:96]`

In [16]:
# Рекуррентная нейросеть
class LSTM(tf.keras.Model):
    def __init__(self, h_size):
        super().__init__()
        self.h_size = h_size
        self.lstm_cell = LSTMCell(h_size)
        

    def call(self, x_all):
        batch, length, emb_size = x.shape
        h = tf.zeros((batch, self.h_size))
        c = tf.zeros((batch, self.h_size))
        h_all = []
        
        for i in range(length):
            h, c = self.lstm_cell(x_all[:, i, :], h, c)
            #
            h_all.append(h)
            
        h_all = tf.transpose(tf.stack(h_all), [1, 0, 2])
        return h_all
    
lstm_my = LSTM(H_SIZE)

In [17]:
y = lstm_my(x)
print(y.shape)

(2, 100, 32)


In [18]:
# Перед тем, как что-то присваивать в параметры модели, нужно чтобы они создались.
# Для этого можно вызвать либо инференс с каким-то входом, либо model.build(...)

lstm_my.lstm_cell.fcXI.kernel = lstm.weights[0][:, 0:32]
lstm_my.lstm_cell.fcXF.kernel = lstm.weights[0][:, 32:64]
lstm_my.lstm_cell.fcXC.kernel = lstm.weights[0][:, 64:96]
lstm_my.lstm_cell.fcXO.kernel = lstm.weights[0][:, 96:128]

lstm_my.lstm_cell.fcHI.kernel = lstm.weights[1][:, 0:32]
lstm_my.lstm_cell.fcHF.kernel = lstm.weights[1][:, 32:64]
lstm_my.lstm_cell.fcHC.kernel = lstm.weights[1][:, 64:96]
lstm_my.lstm_cell.fcHO.kernel = lstm.weights[1][:, 96:128]

lstm_my.lstm_cell.fcXI.bias = lstm.weights[2][0:32]
lstm_my.lstm_cell.fcXF.bias = lstm.weights[2][32:64]
lstm_my.lstm_cell.fcXC.bias = lstm.weights[2][64:96]
lstm_my.lstm_cell.fcXO.bias = lstm.weights[2][96:128]

y = lstm(x)
y_my = lstm_my(x)

print(np.max(np.abs(y.numpy() - y_my.numpy())))

1.4901161e-07
